In [1]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq

In [2]:
# 단계 1 : 문서 로드
loader = PyMuPDFLoader(r"C:\LangGraph\정보처리기사.pdf")
docs = loader.load()
print(f"문서의 페이지수: {len(docs)}")

문서의 페이지수: 11


In [3]:
# 단계 2 : 문서 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)
print(f"분할된 청크의 수 : {len(split_documents)}")

분할된 청크의 수 : 22


In [4]:
# 단계 3 : 임베딩 생성
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3", model_kwargs={"device": "cuda"}, encode_kwargs={"normalize_embeddings": True})

c:\LangGraph\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# 단계 4 : DB 생성 및 저장
vectorstore = Chroma.from_documents(documents=split_documents, embedding=embeddings)

In [8]:
# 단계 5: 검색기(Retriever) 생성
retriever = vectorstore.as_retriever()

In [9]:
# 검색기에 쿼리를 날려 검색된 chunk 결과를 확인
retriever.invoke("OLTP란?")

[Document(metadata={'author': 'Jaehun Jo', 'creationdate': '2021-07-09T16:11:41+09:00', 'creator': 'Microsoft® Word 2016', 'file_path': 'C:\\LangGraph\\정보처리기사.pdf', 'format': 'PDF 1.7', 'keywords': '', 'moddate': '2021-07-09T16:11:41+09:00', 'page': 7, 'producer': 'Microsoft® Word 2016', 'source': 'C:\\LangGraph\\정보처리기사.pdf', 'subject': '', 'title': '', 'total_pages': 11, 'trapped': ''}, page_content='⚫ WAN: 전송 거리가 넓고 라우팅 알고리즘이 필요한 광대역 네트워크(vs LAN) \n⚫ 백본망(Backbone Network): 각기 다른 LAN이나 부분망 간에 정보를 교환하기 위한 경로를 제공하는 망 \n⚫ OSI 7계층: 국제표준화기구(ISO)에서 개발, 네트워크 프로토콜 디자인과 통신을 계층으로 나눠 설명한 것 \n⚫ 1. 물리: 실제 장치들 연결 – 비트 - 리피터(입력 신호 증폭), 허브(가까운 컴퓨터들을 연결) \n⚫ 2. 데이터링크: 각종 흐름, 순서 제어 – 프레임 - 브리지(LAN 연결), 스위치(큰 LAN 생성) – PPP, Ethernet \n⚫ 3. 네트워크: 라우팅, 혼잡 제어 – 패킷 – 라우터(경로 설정) – IP, IGMP, BGP, ARP, RARP \n⚫ 4. 전송: 종단 시스템 간 투명한 데이터 전송 – 세그먼트 – 게이트웨이(서로 다른 프로토콜) - TCP, UDP \n⚫ 5. 세션: 동기점(오류가 있는 데이터의 회복), 대화 제어자 – RPC, SSL/TLS(인증/암호화/무결성) \n⚫ 6. 표현: 통신에 적당한 형태로 변환, 데이터의 암호화/복호화 – SSL/TLS, JPEG \n⚫ 7. 응용: 사용자에

In [10]:
# 단계 6: 프로프트 생성
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Question: 
{question} 
#Context: 
{context} 

#Answer:"""
)

In [11]:
# 단계 7: 언어 모델 생성
llm=ChatGroq(temperature=0, model="gemma2-9b-it", api_key="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")

In [22]:
llm.invoke("트랜잭션에 대해 설명해줘?")

AIMessage(content='## 트랜잭션 (Transaction) 에 대해 알려드릴게요!\n\n트랜잭션은 **데이터베이스에서 여러 개의 작업을 하나의 단위로 처리하는 것**을 의미합니다. \n\n마치 은행에서 여러 개의 계좌 이체를 한 번에 처리하는 것처럼, 트랜잭션은 여러 작업을 하나의 단위로 처리하여 **데이터의 일관성과 정확성을 보장**합니다.\n\n**트랜잭션의 핵심은 ACID 특성**입니다.\n\n* **Atomicity (원자성):** 트랜잭션 내 모든 작업이 성공적으로 완료되면 전체 트랜잭션이 성공으로 처리되고, 어떤 작업이라도 실패하면 전체 트랜잭션이 실패로 처리됩니다. 즉, 완전히 성공하거나 완전히 실패하는 것입니다.\n* **Consistency (일관성):** 트랜잭션이 실행되기 전과 후에 데이터베이스의 일관성이 유지됩니다. 즉, 트랜잭션이 실행된 후 데이터베이스는 일관된 상태를 유지합니다.\n* **Isolation (격리성):** 동시에 실행되는 여러 트랜잭션이 서로 영향을 주지 않습니다. 각 트랜잭션은 독립적으로 실행되며, 다른 트랜잭션의 변경 사항을 보지 못합니다.\n* **Durability (지속성):** 트랜잭션이 성공적으로 완료되면 변경 사항은 영구적으로 데이터베이스에 저장됩니다. 시스템이 오류나 재부팅을 겪더라도 변경 사항은 유지됩니다.\n\n**트랜잭션의 예시:**\n\n* **온라인 쇼핑몰에서 상품 구매:** 상품 주문, 결제, 재고 업데이트 등 여러 작업이 하나의 트랜잭션으로 처리되어, 모든 작업이 성공적으로 완료되거나 실패하면 전체 트랜잭션이 취소됩니다.\n* **은행 계좌 이체:** 출금 계좌에서 금액을 차감하고, 입금 계좌에 금액을 추가하는 작업이 하나의 트랜잭션으로 처리되어, 두 계좌의 잔액이 일관성을 유지합니다.\n\n**트랜잭션은 데이터베이스 시스템에서 매우 중요한 개념**이며, 데이터의 정확성, 일관성, 안정성을 보장하는 데 필수적인 요소입니다.\n\n\n', additional_kwargs={}, 

In [12]:
# 단계 8: 체인(chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [21]:
question = "트랜잭션에 대해 설명해줘?"
response = chain.invoke(question)
print(response)

DB시스템에서 하나의 논리적인 기능을 수행하기 위한 작업의 기본 단위입니다. 



